In [68]:
import pandas as pd

<ipython-input-68-a7a5c25487ab>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  bx_users = pd.read_csv("/content/drive/MyDrive/Books/BX-Users.csv", encoding='latin-1', delimiter=';', error_bad_lines=False)


In [268]:
#read in data frames data from ".csv" file specifying the delimiter and the encoding type and we jump over the lines with bad data 

bx_users = pd.read_csv("/content/drive/MyDrive/Books/BX-Users.csv", encoding='latin-1', delimiter=';', error_bad_lines=False)
bx_book_ratings = pd.read_csv("/content/drive/MyDrive/Books/BX-Book-Ratings.csv", encoding='latin-1', delimiter=';', error_bad_lines=False)
bx_books = pd.read_csv("/content/drive/MyDrive/Books/BX-Books.csv", encoding='latin-1', delimiter=';', error_bad_lines=False)

#we split the column "Location" in bx_users to have access easier to "bx_users" data 
location_columns = bx_users['Location'].str.split(',', expand=True)
bx_users[['City', 'Region', 'Country']] = location_columns[[0,1,2]]
bx_users.drop(columns=['Location'], inplace=True)
display(bx_users)

Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, saw 10
Skipping line 245933: expected 8 fields, saw 9
Skipping line 251296: expected 8 fields, saw 9
Skipping line 259941: expected 8 fields, saw 9
Skipping line 261529: expected 8 fields, saw 9



,User-ID,Age,City,Region,Country
0,1,NaN,nyc,new york,usa
1,2,18.0,stockton,california,usa
2,3,NaN,moscow,yukon territory,russia
3,4,17.0,porto,v.n.gaia,portugal
4,5,NaN,farnborough,hants,united kingdom
...,...,...,...,...,...
278853,278854,NaN,portland,oregon,usa
278854,278855,50.0,tacoma,washington,united kingdom
278855,278856,NaN,brampton,ontario,canada
278856,278857,NaN,knoxville,tennessee,usa


In [288]:
# we make an inner join between the three data_frames
merged_data = pd.merge(bx_users, bx_book_ratings, left_on = 'User-ID', right_on = 'User-ID')
merged_data = pd.merge(merged_data, bx_books, left_on = 'ISBN', right_on = 'ISBN' )
# we work with perfect data ( records that don't have null values )

merged_data = merged_data.dropna()
merged_data.drop(merged_data[merged_data['Country'] == ''].index, inplace = True)

# we take just the books that have a perfect rating ( perfect rating = 10)
merged_data['Book-Rating'] = merged_data['Book-Rating'].astype(int)
merged_data = merged_data.loc[merged_data['Book-Rating'] == 10]

merged_data = merged_data.rename(columns = {'User-ID':'NO of perfect rating'})

# we group the records to have a better undestanding which books are the most popular and we order them 
#by their number of pefect ratings 
merged_data = merged_data.groupby(['Country', 'Book-Title', 'Book-Author', 'ISBN', 'Publisher','Year-Of-Publication'],as_index=False)['NO of perfect rating'].count()
merged_data = merged_data.sort_values(by = ['NO of perfect rating'], ascending = False).reset_index(drop = True)

# if we need to see the most popular book in a specific country we make a call to this function with the country we want 
def country_wanted(country):
  filtered_data = merged_data[merged_data['Country'].str.contains(f'{country}', regex=True, na=False)]
  return filtered_data

merged_data = country_wanted('usa')
display(merged_data)

,Country,Book-Title,Book-Author,ISBN,Publisher,Year-Of-Publication,NO of perfect rating
0,usa,The Da Vinci Code,Dan Brown,0385504209,Doubleday,2003,100
1,usa,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,059035342X,Arthur A. Levine Books,1999,97
2,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,043935806X,Scholastic,2003,79
3,usa,The Lovely Bones: A Novel,Alice Sebold,0316666343,"Little, Brown",2002,75
4,usa,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,0439139597,Scholastic,2000,63
...,...,...,...,...,...,...,...
37544,usa,A Pen Warmed-Up in Hell; Mark Twain in Protest.,"Samuel Langhorne, Clemens",0060906782,Harpercollins,1979,1
37545,usa,A Patchwork Planet,Anne Tyler,037540256X,Random House Inc,1998,1
37546,usa,A Past in Hiding: Memory and Survival in Nazi ...,Mark Roseman,0805063269,Metropolitan Books,2001,1
37547,usa,A Passionate Magic (Heartspell),Flora Speer,0505524392,Love Spell,2001,1


In [278]:
import requests
import pandas as pd
import requests
#for the depricated append 
import warnings
warnings.filterwarnings('ignore')

# in order to get the description of the books we will use the ISBN of the book 
# The International Standard Book Number (ISBN) is a numeric commercial book identifier
# that is intended to be unique. Publishers purchase or receive ISBNs from an affiliate of the International ISBN Agency.
def get_book_description(isbn):
    url = f'https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}'
    response = requests.get(url)
    data = response.json()

    if 'items' in data and len(data['items']) > 0:
        book_info = data['items'][0]['volumeInfo']
        description = book_info.get('description', 'No description available')
        return description
    else:
        return 'Book not found'
# because there are so manny books in our data we can set for 
# how manny books we want a description 
def get_description(number_of_books):
  # List of ISBNs
  list_of_ISBN = merged_data['ISBN'].values.tolist()
  list_of_ISBN = list_of_ISBN[:number_of_books]
  # Create an empty DataFrame
  book_description = pd.DataFrame(columns=['ISBN', 'Description'])

  # Iterate over the ISBNs and fetch the descriptions
  for isbn in list_of_ISBN:
      description = get_book_description(isbn)
      book_description = book_description.append({'ISBN': isbn, 'Description': description}, ignore_index=True)

  # we exclude the books for which we don't have any description
  book_description.drop(book_description[book_description['Description'] == 'Book not found'].index, inplace = True)
  return book_description
   
books = get_description(20)  
display(books)

,ISBN,Description
0,0385504209,Harvard symbologist Robert Langdon and French ...
1,059035342X,Rescued from the outrageous neglect of his aun...
2,043935806X,Collects the complete series that relates the ...
3,0316666343,Sebold's mesmerizing and luminous first novel-...
4,0439139597,Collects the complete series that relates the ...
5,0446310786,The unforgettable novel of a childhood in a sl...
6,0142001740,The multi-million bestselling novel about a yo...
7,0439136350,During his third year at Hogwarts School for W...
8,0786868716,Eddie dies on his eighty-third birthday in a t...
9,0590353403,Rescued from the outrageous neglect of his aun...


In [279]:
# we make an inner join between the data frame with desciptions (books) and the data frame with books (merged_data) 
books_with_descript = pd.merge(books, merged_data, left_on = 'ISBN', right_on = 'ISBN')
books_with_descript = books_with_descript.sort_values(by =  'Country')
books_with_descript.drop(books_with_descript[books_with_descript['Country'] == ''].index, inplace = True)
books_with_descript = books_with_descript.sort_values(by = 'NO of perfect rating', ascending = False )
# at the end we will get the data frame with the most popular books and their description 
display(books_with_descript)

,ISBN,Description,Country,Book-Title,Book-Author,Publisher,Year-Of-Publication,NO of perfect rating
0,0385504209,Harvard symbologist Robert Langdon and French ...,usa,The Da Vinci Code,Dan Brown,Doubleday,2003,100
1,059035342X,Rescued from the outrageous neglect of his aun...,usa,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,Arthur A. Levine Books,1999,97
2,043935806X,Collects the complete series that relates the ...,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,Scholastic,2003,79
3,0316666343,Sebold's mesmerizing and luminous first novel-...,usa,The Lovely Bones: A Novel,Alice Sebold,"Little, Brown",2002,75
4,0439139597,Collects the complete series that relates the ...,usa,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,Scholastic,2000,63
5,0446310786,The unforgettable novel of a childhood in a sl...,usa,To Kill a Mockingbird,Harper Lee,Little Brown &amp; Company,1988,60
6,0142001740,The multi-million bestselling novel about a yo...,usa,The Secret Life of Bees,Sue Monk Kidd,Penguin Books,2003,58
7,0439136350,During his third year at Hogwarts School for W...,usa,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,Scholastic,1999,56
8,0786868716,Eddie dies on his eighty-third birthday in a t...,usa,The Five People You Meet in Heaven,Mitch Albom,Hyperion,2003,55
9,0590353403,Rescued from the outrageous neglect of his aun...,usa,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,Scholastic,1998,52


In [283]:
book_authors = []
description = []

# Iterate over the authors
lst = merged_data['Book-Author'].values.tolist()
lst = lst[:20]
for author in lst:
        # Make a request to the Wikipedia API for the author's infos
        response = requests.get(f"https://en.wikipedia.org/api/rest_v1/page/summary/{author}")
        #  if the HTTP response status code is 200, which indicates a successful request
        if response.status_code == 200:
            data = response.json()
            summary = data.get('extract', 'Description not available')
            # we add the name of author and its description to the list 
            book_authors.append(author)
            description.append(summary)
        else:
            # Add the author's name and a message in case we do not find a description 
            book_authors.append(author)
            description.append('Description not available')

authors_frame = pd.DataFrame({'Author': book_authors, 'Description': description})
authors_frame.drop(authors_frame[authors_frame['Description'] == 'Description not available'].index, inplace = True)
authors_frame = authors_frame.drop_duplicates()
display(authors_frame)

,Country,Book-Title,Book-Author,ISBN,Publisher,Year-Of-Publication,NO of perfect rating
0,usa,The Da Vinci Code,Dan Brown,0385504209,Doubleday,2003,100
1,usa,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,059035342X,Arthur A. Levine Books,1999,97
2,usa,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,043935806X,Scholastic,2003,79
3,usa,The Lovely Bones: A Novel,Alice Sebold,0316666343,"Little, Brown",2002,75
4,usa,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,0439139597,Scholastic,2000,63
...,...,...,...,...,...,...,...
37544,usa,A Pen Warmed-Up in Hell; Mark Twain in Protest.,"Samuel Langhorne, Clemens",0060906782,Harpercollins,1979,1
37545,usa,A Patchwork Planet,Anne Tyler,037540256X,Random House Inc,1998,1
37546,usa,A Past in Hiding: Memory and Survival in Nazi ...,Mark Roseman,0805063269,Metropolitan Books,2001,1
37547,usa,A Passionate Magic (Heartspell),Flora Speer,0505524392,Love Spell,2001,1


In [292]:
popular_authors = merged_data['Book-Author'].value_counts().reset_index()
display(popular_authors)

,index,Book-Author
0,Stephen King,163
1,Nora Roberts,144
2,William Shakespeare,97
3,Anne McCaffrey,79
4,Piers Anthony,69
...,...,...
13082,Tess Fries,1
13083,GRAHAM SWIFT,1
13084,Lorraine Dopson,1
13085,Donna Gillespie,1
